In [2]:
import pandas as pd
import numpy as np
from scipy import sparse

from pattern.text.en import suggest,Sentence
from nltk.tokenize import word_tokenize


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
movie_user_searched="hak ridge"
User_rated_list = [("Alice in Wonderland",3),("Aliens",1),("2001: A Space Odyssey",2)]

In [39]:
#def spell_checker(movie_user_searched):
l=word_tokenize(movie_user_searched)
# for x in l:
#     print("suggestion:",suggest(x))
# #sent= movie_user_searched
#sent = print("Sentence:",str(Sentence(l)))

# print("Sentence:",sent)


AttributeError: 'list' object has no attribute 'split'

In [4]:
content_df = pd.read_csv("C:\\Users\\aayus\\Desktop\\PROJECT EXIBHITION\\movie recommendation\\movie_dataset.csv")
collab_df=pd.read_csv("C:\\Users\\aayus\\Desktop\\PROJECT EXIBHITION\\movie recommendation\\dataset\\movies.csv")
ratings = pd.read_csv("C:\\Users\\aayus\\Desktop\\PROJECT EXIBHITION\\movie recommendation\\dataset\\ratings.csv")
collab_df=pd.merge(collab_df,ratings).drop(['genres','timestamp'],axis=1)
collab_df.head()

,movieId,title,userId,rating
0,1,Toy Story,1,4.0
1,1,Toy Story,5,4.0
2,1,Toy Story,7,4.5
3,1,Toy Story,15,2.5
4,1,Toy Story,17,4.5


display both dataframes

In [29]:
print("cont:",content_df.columns)
print("collab:",collab_df.columns)



cont: Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'tagline', 'vote_average',
       'vote_count', 'cast', 'director'],
      dtype='object')
collab: Index(['movieId', 'title', 'userId', 'rating'], dtype='object')


# Content Based

In [30]:
###### helper functions. Use them when needed #######
def get_title_from_index(index):
	return content_df[content_df.index == index]["title"].values[0]

def get_index_from_title(title):
	return content_df[content_df.title == title]["index"].values[0]

In [31]:
features = ['keywords','cast','genres','director','runtime']

# Create a column in DF which combines all selected features

for feature in features:
	content_df[feature] = content_df[feature].fillna('')

def combine_features(row):
	try:
		return row['keywords'] +" "+row['cast']+" "+row["genres"]+" "+row["director"]+" "+row["runtime"]
	except:
		print ("Error:", row)

content_df["combined_features"] = content_df.apply(combine_features,axis=1)

#print ("Combined Features:", content_df["combined_features"].head())

 Create count matrix from this new combined column

In [32]:
cv = CountVectorizer()

count_matrix = cv.fit_transform(content_df["combined_features"])

#print(count_matrix)

In [33]:
# Compute the Cosine Similarity based on the count_matrix
cosine_sim = cosine_similarity(count_matrix) 
print(cosine_sim)

[[1.         0.10182663 0.11572751 ... 0.         0.         0.        ]
 [0.10182663 1.         0.07332356 ... 0.0352235  0.         0.        ]
 [0.11572751 0.07332356 1.         ... 0.         0.10540926 0.        ]
 ...
 [0.         0.0352235  0.         ... 1.         0.         0.04089304]
 [0.         0.         0.10540926 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.04089304 0.         1.        ]]


above array shows movie-movie similarity scores

In [34]:
movie_index = get_index_from_title(movie_user_searched)

similar_movies =  list(enumerate(cosine_sim[int(movie_index)]))


## Step 7: Get a list of similar movies in descending order of similarity score
sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)
#print(sorted_similar_movies)
# Step 8: Print titles of first 50 movies
i=0
content_recommended = []
for element in sorted_similar_movies:
    content_recommended.append(get_title_from_index(element[0]))
    i = i+1
    if i>50:
        break


IndexError: index 0 is out of bounds for axis 0 with size 0

In [26]:
content_recommended_df= pd.DataFrame(content_recommended)
print("Content based Recommendation for movie:",movie_user_searched)
content_recommended_df

Content based Recommendation for movie: Avengers: Age of Ultron


,0
0,Avengers: Age of Ultron
1,The Avengers
2,Iron Man 2
3,Captain America: Civil War
4,Captain America: The Winter Soldier
5,Thor: The Dark World
6,Ant-Man
7,X-Men
8,X2
9,The Incredible Hulk


# Collaborative Based

In [5]:
print(collab_df.shape)
collab_df



(100836, 4)


,movieId,title,userId,rating
0,1,Toy Story,1,4.0
1,1,Toy Story,5,4.0
2,1,Toy Story,7,4.5
3,1,Toy Story,15,2.5
4,1,Toy Story,17,4.5
...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic,184,4.0
100832,193583,No Game No Life: Zero,184,3.5
100833,193585,Flint,184,3.5
100834,193587,Bungo Stray Dogs: Dead Apple,184,3.5


In [6]:
userRatings = collab_df.pivot_table(index=['userId'],columns=['title'],values='rating')
userRatings.head()

title,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,*batteries not included,...,Zulu,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
userRatings = userRatings.dropna(thresh=6, axis=1).fillna(0,axis=1)
userRatings.head()

title,"'burbs, The",(500) Days of Summer,*batteries not included,10 Cloverfield Lane,10 Things I Hate About You,"10,000 BC",101 Dalmatians,101 Dalmatians (One Hundred and One Dalmatians),102 Dalmatians,12 Angry Men,...,Zero Dark Thirty,Zero Effect,Zodiac,Zombieland,Zoolander,Zootopia,[REC],eXistenZ,xXx,¡Three Amigos!
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head(100)

title,"'burbs, The",(500) Days of Summer,*batteries not included,10 Cloverfield Lane,10 Things I Hate About You,"10,000 BC",101 Dalmatians,101 Dalmatians (One Hundred and One Dalmatians),102 Dalmatians,12 Angry Men,...,Zero Dark Thirty,Zero Effect,Zodiac,Zombieland,Zoolander,Zootopia,[REC],eXistenZ,xXx,¡Three Amigos!
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The",1.000000,0.063117,0.235908,-0.023768,0.143482,0.011998,0.087931,0.224052,-0.018608,0.034133,...,0.032470,0.134701,0.153158,0.101301,0.049897,0.003233,-0.017905,0.187953,0.062174,0.353194
(500) Days of Summer,0.063117,1.000000,0.133949,0.142471,0.273989,0.193960,0.148903,0.142141,0.066567,0.160679,...,0.178655,0.068407,0.414585,0.355723,0.252226,0.216007,0.126147,0.053614,0.241092,0.125905
*batteries not included,0.235908,0.133949,1.000000,0.035596,0.061144,-0.017106,0.073459,0.106100,-0.012561,0.026313,...,0.058327,0.039055,0.194530,0.121010,0.071852,-0.024573,-0.012086,0.115396,-0.000060,0.234514
10 Cloverfield Lane,-0.023768,0.142471,0.035596,1.000000,-0.005799,0.112396,0.006139,-0.016835,-0.017692,0.031619,...,0.099059,-0.023477,0.272347,0.241751,0.195054,0.319371,0.082246,0.177846,0.096638,0.002733
10 Things I Hate About You,0.143482,0.273989,0.061144,-0.005799,1.000000,0.244670,0.223481,0.211473,0.109729,0.013083,...,0.104858,0.132460,0.091853,0.158637,0.281934,0.050031,0.088391,0.121029,0.130813,0.110612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Age of Innocence, The",0.136564,0.051691,0.057659,0.018431,0.148320,0.024920,0.134095,0.140306,-0.017927,0.101469,...,0.051461,0.201522,0.115915,0.047371,0.095380,0.043483,0.032087,0.314265,0.141280,0.107412
"Aguirre: The Wrath of God (Aguirre, der Zorn Gottes)",0.093271,0.034652,-0.012160,-0.017127,-0.033931,0.049846,-0.002414,0.023784,-0.013409,0.165062,...,-0.017236,-0.017794,0.131585,0.010220,0.012183,0.072171,-0.012902,0.054643,0.054684,0.033507
Air America,0.143746,0.063197,0.105972,0.058942,0.014007,0.031532,-0.025677,0.022744,-0.010920,0.003947,...,0.161160,0.142816,0.089143,0.079184,0.061499,0.027895,-0.010508,0.041778,0.068382,0.311347


In [16]:
def get_similar(title,rating):
    similar_movies = corrMatrix[title]*(rating-2.25)
    similar_movies = similar_movies.sort_values(ascending=False)
    return similar_movies

In [21]:
collab_recommended_df = pd.DataFrame()
for movie,rating in User_rated_list:
    collab_recommended_df = collab_recommended_df.append(get_similar(movie,rating),ignore_index = True)
    


collab_recommended_df = collab_recommended_df.sum().sort_values(ascending=False).head(50)
collab_recommended_df

Alice in Wonderland                                   0.519325
Sweeney Todd: The Demon Barber of Fleet Street        0.234826
Tangled                                               0.234563
The Theory of Everything                              0.234540
Peter Pan                                             0.228494
Golden Compass, The                                   0.219670
Step Brothers                                         0.218168
The Imitation Game                                    0.217646
Angels & Demons                                       0.214396
Pocahontas                                            0.212052
Megamind                                              0.205567
Aristocats, The                                       0.199302
Chronicles of Narnia: Prince Caspian, The             0.197475
Wedding Planner, The                                  0.196080
Percy Jackson & the Olympians: The Lightning Thief    0.191709
Ugly Truth, The                                       0